In [ ]:
Data = reshape( BINRAW, 64, 64, nFrs );
( J, K, L )  = size( Data );
# We apply a Temporal Gaussian smoothing ( this greatly affects the animations )
BINSMOOTH = zeros( J, K, L );
for j = 1 : J, k = 1 : K
    BINSMOOTH[ j, k, : ] = GaussSmoothTemporal( vec( Data[ j, k, : ] ), 3, "I" );
end

function GaussSmoothTemporal( Datos::Array, Sigma = 3, Gauss = "I" )
    # Un suavizado Gaussiano temporal.
    # Esto es escencialmente un filtro pasabajos.
    # Depende implicitamente de la frecuencia de muestreo.
    # sigma esta medido en pixeles, es la desviacion estandar de nuestro kernel.
    # El A de nuestra ventana seran 3 * sigma
    A = ceil( Sigma * 3 );
    B = ones( A );
    result = zeros( size( Datos ) );
    datosB = vcat( B * Datos[ 1 ], Datos, B * Datos[ end ] );
    if Gauss == "K"
        kernel = map( x -> UnNormGaussK( x, Sigma ), collect( -A : A ) );
        kernel = kernel / ( sum( kernel ) );
    elseif Gauss == "I"
        kernel = map( x -> UnNormGaussI( x, Sigma ), collect( -A : A ) );
        kernel = kernel / ( sum( kernel ) );
    end
    for t = ( A + 1 ) : ( length( Datos ) + A )
        result[ t - A ] = sum(
            datosB[ ( t - A ) : ( t + A ) ] .* kernel );
    end
    return result
end
function UnNormGaussI( x, sigma )
    cte = ( 1 / ( sigma * sqrt( 2 * pi ) ) );
    return cte * exp( -( ( x ^ 2 ) / ( 2 * ( sigma ^ 2 ) ) ) );
end

Data = reshape( Data, nChs, nFrs );

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Parameters = Dict(
    # for detecting saturations
    "maxvolt"              => 3000, # μV
    "minvolt"              => -3000, # μV
    # for the neighborhood
    "minchannels"          => 8, # needed valid channels on the neighborhood
    "maxradius"            => 4, # maximus radius for the searching neighborhood ( 1 -> 8)
    # how much of the channel can we tolerate being saturated before patching it all
    "limite_saturacion"    => 0.4, # [ 0, 1 ]
    "window"               => 5, # ms, para la deteccion de eventos
    "bit"                  => 1, # ms, deslizamiento de la ventana
    "stokes"               => 0.3, # ms, distancia maxima entre eventos para ser descartados
    "thr"                  => -50, # μV, umbral estatico
    "ΔT"                   => 250, # ms, para calcular el cambio de voltaje
    "MUAlF"                => 290, # Hz, limite inferior de frecuencia para el MUA
    "MUAHF"                => floor(Int,(Variables[ "SamplingRate" ]/2)) - 10,
    # Hz, limite superior de frecuencia para el MUA
    "MUAfac"               => 10, # Hz, banda de transicion para el MUA
    "LFPlF"                => 2, # Hz, limite inferior de frecuencia para el LFP
    "LFPHF"                => 302, # Hz, limite superior de frecuencia para el LFP
    "LFPfac"               => 2, # Hz, banda de transicion para el LFP
    "SamplingRate"         => Variables[ "SamplingRate" ]
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
MUAlF = Parameters[ "MUAlF" ];
MUAHF = Parameters[ "MUAHF" ];
MUAfac = Parameters[ "MUAfac" ];
data = round.( MatrixFilter( Parameters, Data, MUAlF, MUAHF, MUAfac, "BPF" ), digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

datapre = round.( MatrixFilter( Parameters, BINRAWPRE, MUAlF, MUAHF, MUAfac, "BPF" ), digits = 3 );
datapos = round.( MatrixFilter( Parameters, BINRAWPOS, MUAlF, MUAHF, MUAfac, "BPF" ), digits = 3 );

SpikeIndexes = GetIndexes( Parameters, data, "Dynamic", "Windows", Empties );

using StatsBase

nChs, nFrs = size( BINRAW );

pre = 2; # ms
pos = 3; # ms
Fpre = ms2Frs( Parameters, pre );
Fpos = ms2Frs( Parameters, pos );
RightLimit = nFrs;
LeftLimit = 1;
SpikesSize = Fpre + Fpos + 1;

All_Spikes = [ ];
All_Average = [ ];
All_Deviation = [ ];
All_NoAverage = [ ];
All_NoDeviation = [ ];

# CutSpikes
for ch = 1:nChs
    ChannelIndexes = SpikeIndexes[ ch ];
    RightPass = findall( ( ChannelIndexes .+ Fpos ) .> RightLimit );
    LeftPass = findall( ( ChannelIndexes .- Fpre ) .< LeftLimit );
    nSpikes = length( ChannelIndexes );
    SpikesVoltage = zeros( Float64, nSpikes, ( Fpre + Fpos + 1 ) );
    OKnSpikes = setdiff( setdiff( 1:nSpikes, RightPass ), LeftPass );
    if !isempty( RightPass )
        for rp in RightPass
            A = ChannelIndexes[ rp ];
            fits = length( ( A - Fpre ) : RightLimit );
            SpikesVoltage[ rp, ( 1 : fits ) ] = data[ ch, ( ( A - Fpre ) : RightLimit ) ];
            lack = length( SpikesVoltage[ rp, ( ( fits + 1 ) : end ) ]);
            SpikesVoltage[ rp, ( ( fits + 1 ) : end ) ] = datapos[ ch, ( 1 : lack ) ];
        end
    end
    if !isempty( LeftPass )
        for lp in LeftPass
            A = ChannelIndexes[ lp ];
            I = abs( A - Fpre ) + 1;
            SpikesVoltage[ lp,  ( I + 1 ) : end ] = data[ ch, ( LeftLimit : ( A + Fpos ) ) ];
            SpikesVoltage[ lp, ( 1 : I ) ] = datapre[ ch, ( ( end - I + 1 ) : end ) ];
        end
    end
    for nSpk in OKnSpikes
        A = ChannelIndexes[ nSpk ];
        SpikesVoltage[ nSpk, : ] = data[ ch, ( A - Fpre ) : ( A + Fpos ) ];
    end
#     Average = mean( SpikesVoltage, dims = 2 );
#     Deviation = std( SpikesVoltage, dims = 2 );
#     NoAverage = findall( vec( abs.( Average ) ) .< 1 );
#     NoDeviation = findall( vec( abs.( Deviation ) ) .< 3 );
    push!( All_Spikes, SpikesVoltage );
#     push!( All_Average, Average );
#     push!( All_Deviation, Deviation );
#     push!( All_NoAverage, NoAverage );
#     push!( All_NoDeviation, NoDeviation );
end

pruebas = deepcopy( SpikeIndexes );

for a = 1:4096
    NOs = sort( unique( union( All_NoAverage[ a ], All_NoDeviation[ a ] ) ) );
    if !isempty( NOs )
        deleteat!( pruebas[ a ], NOs )
    end
end

aux01 = length.( SpikeIndexes );
aux02 = length.( pruebas );
a,b = extrema( RemoveInfs( log.( aux01 ) ) )
A = Zplot( RemoveInfs( log.( aux01 ) ), "W", :bluesreds );
B = Zplot( RemoveInfs( log.( aux02 ) ), "W", :bluesreds );
plot( A, B, layout = (1,2), wsize = (800, 400) )

All_Spikes_Pruebas = [ ];
# CutSpikes
for ch = 1:nChs
    ChannelIndexes = pruebas[ ch ];
    RightPass = findall( ( ChannelIndexes .+ Fpos ) .> RightLimit );
    LeftPass = findall( ( ChannelIndexes .- Fpre ) .< LeftLimit );
    nSpikes = length( ChannelIndexes );
    SpikesVoltagePruebas = zeros( Float64, nSpikes, ( Fpre + Fpos + 1 ) );
    OKnSpikes = setdiff( setdiff( 1:nSpikes, RightPass ), LeftPass );
    if !isempty( RightPass )
        for rp in RightPass
            A = ChannelIndexes[ rp ];
            fits = length( ( A - Fpre ) : RightLimit );
            SpikesVoltagePruebas[ rp, ( 1 : fits ) ] = data[ ch, ( ( A - Fpre ) : RightLimit ) ];
            lack = length( SpikesVoltagePruebas[ rp, ( ( fits + 1 ) : end ) ]);
            SpikesVoltagePruebas[ rp, ( ( fits + 1 ) : end ) ] = datapos[ ch, ( 1 : lack ) ];
        end
    end
    if !isempty( LeftPass )
        for lp in LeftPass
            A = ChannelIndexes[ lp ];
            I = abs( A - Fpre ) + 1;
            SpikesVoltagePruebas[ lp,  ( I + 1 ) : end ] = data[ ch, ( LeftLimit : ( A + Fpos ) ) ];
            SpikesVoltagePruebas[ lp, ( 1 : I ) ] = datapre[ ch, ( ( end - I + 1 ) : end ) ];
        end
    end
    for nSpk in OKnSpikes
        A = ChannelIndexes[ nSpk ];
        SpikesVoltagePruebas[ nSpk, : ] = data[ ch, ( A - Fpre ) : ( A + Fpos ) ];
    end
    push!( All_Spikes_Pruebas, SpikesVoltagePruebas );
end

x = 8
plot( plot(All_Spikes[x]'),plot(All_Spikes_Pruebas[x]'),
layout = ( 2,1 ), wsize = (800, 600), leg = :none
)

All_Spikes[8][]






X = deepcopy( SpikeIndexes );

for a = 1:nChs
    deleteat!(X[a], All_NoDeviation[a] )



function cut_spikes( channel::Array, index::Array, w_pre::Int, w_post::Int )
    index = index[ Bool.( 1 .- ( ( index .+ w_post ) .> length( channel ) ) ) ];
    index = index[ Bool.( 1 .- ( ( index .- w_pre ) .< 1 ) ) ];
    spikes = zeros( length( index ), ( w_pre + w_post + 1) );
    lim1 = index .- w_pre; lim2 = index .+ w_post;
    for i = 1:length( index )
        spikes[ i, : ] = channel[ lim1[ i ]:lim2[ i ] ];
    end
    return spikes
end


function centrar_espigas( spikes::Array, limite_izquierdo::Int, limite_derecho::Int )
    frames_centros = zeros( Int, size( spikes, 2 ) );
    for i = 1:size( spikes, 2 )
        frames_centros[ i ] = findall( minimum( spikes, dims = 2 )[ i ] .== spikes[ i, : ] )[ ];
    end
    moda = StatsBase.mode( frames_centros )
    limite_derecho = size( spikes, 2 ) - limite_derecho;
    espigas_movidas = findall( frames_centros .!= moda );
    por_corregir = espigas_movidas[ limite_izquierdo .< frames_centros[ espigas_movidas ] .< limite_derecho ];
    cuanto_corregir =
        frames_centros[
            espigas_movidas ][ limite_izquierdo .< frames_centros[ espigas_movidas ] .< limite_derecho ];
    retrasar = por_corregir[ ( cuanto_corregir .- moda ) .> 0 ];
    adelantar = por_corregir[ ( cuanto_corregir .- moda ) .< 0 ];
    if !isempty( retrasar )
        cuanto_retrasar = frames_centros[ retrasar ] .- moda;
        for i = 1:length( retrasar )
            spikes[ retrasar[ i ], : ] = circshift( spikes[ retrasar[ i ], : ], ( -1 * cuanto_retrasar[ i ] ) );
        end
    end
    if !isempty( adelantar )
        cuanto_adelantar = frames_centros[ adelantar ] .- moda;
        for i = 1:length( adelantar )
            spikes[ adelantar[ i ], : ] = circshift( spikes[ adelantar[ i ], : ], ( -1 * cuanto_adelantar[ i ] ) );
        end
    end
    return spikes
end



In [1]:
# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
@suppress begin
    # PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
    PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# FILEBRW = "/run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10.brw";
FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Hipocampo\BRWs\Phase_10.brw";
FILEVARS = FindContent( "Variables.jld", FILEBRW );
Variables = LoadDict( FILEVARS );
FILESINDEXES = FindContent( "SpikeIndexes",  FILEBRW );
FILESVOLTAGE = FindContent( "Voltage",  FILEBRW );
PATHSPIKES = replace( dirname( FILESVOLTAGE[ 1 ] ), "Voltage" => "Spikes" ); 
mkpath( PATHSPIKES );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Parameters = Dict(
    "MUAlF"                => 290, # Hz, limite inferior de frecuencia para el MUA
    "MUAHF"                => 5010, # Hz, limite superior de frecuencia para el MUA
    "MUAfac"               => 10, # Hz, banda de transicion para el MUA
    "SamplingRate"         => Variables[ "SamplingRate" ]
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
w_pre = 1; #
w_post = 2; #
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

FILESPIKES = raw"C:\Users\Siby\Desktop\Datos\Hipocampo\Phase_10\BinSpikes\BIN16.jld";
aux = load( FILESPIKES );
Spikes = aux[ "Spikes" ];
Indexes = aux[ "Indexes" ];

ValidChannels = findall( length.( Indexes ) .!= 0 );
BINRAW = LoadDict( raw"C:\Users\Siby\Desktop\Datos\Hipocampo\Phase_10\Voltage\BIN16.jld" );
MUAlF = Parameters[ "MUAlF" ];
MUAHF = Parameters[ "MUAHF" ];
MUAfac = Parameters[ "MUAfac" ];
BINMUA = round.( MatrixFilter( 
        Parameters, Float64.( BINRAW ), MUAlF, MUAHF, MUAfac, "BPF" ), digits = 3 );
SamplingRate = Variables[ "SamplingRate" ];

using MAT
using Dates

right_now = Dates.format( 
    round( Dates.DateTime( Dates.now( ) ), Dates.Minute( 15 ) ), Dates.RFC1123Format );
par = Dict(
    "channels"         => 1,
    "segments_length"  =>( 4 / 60 ),
    "sr"               => SamplingRate,
    "tmax"             => "all",
    "tmin"             => 0,
    "w_pre"            => ms2Frs( Variables, w_pre ),
    "w_post"           => ms2Frs( Variables, w_post ),
    "alignment_window" => ms2Frs( Variables, w_pre ) + 1,
    "stdmin"           => 4,
    "stdmax"           => 35,
    "detect_fmin"      => 290,
    "detect_fmax"      => 5010,
    "sort_fmin"        => 290,
    "sort_fmax"        => 5010,
    "ref_ms"           => 1,
    "detection"        => "neg",
    "int_factor"       => 5,
    "interpolation"    => "y",
    "sort_order"       => 2,
    "detect_order"     =>  Int( floor( ( SamplingRate / 290 ) / 5 ) ) + 1,
    "detection_date"   => right_now
        );

PATHCHANNELS = replace( PATHSPIKES, "Spikes" => "Channels" ); mkpath( PATHCHANNELS );

@time for n in ValidChannels
    chname = joinpath( PATHCHANNELS, string( "ch", lpad( n, 4, "0" ), "_spikes.mat" ) );
    OK = length.( Spikes[ n ] ) .== 55;
    indexes = Indexes[ n ][ OK ];
    spikes = Spikes[ n ][ OK ];
    matwrite( chname,
        Dict(
        "par" => par,
        "threshold" => -50,
        "index" => reshape(
                ( ( indexes ./ SamplingRate ) * 1000 ),
                    1, length( indexes ) ),
        "spikes" => Array( hcat( spikes... )' ),
        "psegment" => BINMUA[ n, : ],
        "sr_psegment" => SamplingRate,
        ); compress = true
        );
end


# for B = 1:4096
#     Ind = SpikesDynamic[ A ][ B ];
#     push!( Indexes, Ind )
#     INI = Ind .- ms2Frs( Parameters, Parameters[ "w_pre" ] );
#     INI[ INI .<= 0 ] .= 1;
#     ENDI = Ind .+ ms2Frs( Parameters, Parameters[ "w_post" ] );
#     ENDI[ ENDI .> size( BIN, 2 ) ] .= size( BIN, 2 );
#     sp = [ ];
#     for ie = 1:length( INI )
#         push!( sp, BIN[ B, INI[ ie ]:ENDI[ ie ] ] );
#     end
#     push!( Spikes, sp )
# end
# FILESPIKES = replace( FILESVOLTAGE[ A ], "Voltage" => "Spikes" );
# save( FILESPIKES, "Spikes", Spikes, "Indexes", Indexes );

N = hcat( SpikesDynamic... );



In [ ]:
using JLD
# PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
push!( LOAD_PATH, PATHFunctions );
using F2023
#FILEBRW = "/run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10.brw";
FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Hipocampo\BRWs\Phase_10.brw";
FILEVARS = FindContent( "Variables.jld", FILEBRW );
Variables = LoadDict( FILEVARS );
FILESINDEXES = FindContent( "SpikeIndexes",  FILEBRW ); 
FILESVOLTAGE = FindContent( "Voltage",  FILEBRW );
PATHSPIKES = replace( dirname( FILESVOLTAGE[ 1 ] ), "Voltage" => "Spikes" ); 
mkpath( PATHSPIKES );

Parameters = Dict(
    "MUAlF"                => 290, # Hz, limite inferior de frecuencia para el MUA
    "MUAHF"                => 5010, # Hz, limite superior de frecuencia para el MUA
    "MUAfac"               => 10, # Hz, banda de transicion para el MUA
    "SamplingRate"         => Variables[ "SamplingRate" ],
    "w_pre" => 1,
    "w_post" => 2
);

SpikesDynamic = [ ];
for α = 1:length( FILESINDEXES );
    aux = LoadDict( FILESINDEXES[ α ] );
    MUASpikes = aux[ 2 ];
    push!( SpikesDynamic, MUASpikes[ 2 ] );
end

nSpikesDynamic = zeros( Int, 4096 );
for α in 1:length( FILESINDEXES );
    nSpikesDynamic = nSpikesDynamic .+ length.( SpikesDynamic[ α ] );
end

SpikeSize = ms2Frs( 
    Parameters, Parameters[ "w_pre" ] ) + ms2Frs( Parameters, Parameters[ "w_post" ] ) + 1;

binsize = Int( Variables[ "NRecFrames" ] / length( FILESVOLTAGE ) );
AllInd = Array{ Any }( undef, 4096 ); fill!( AllInd, [ ] );
for A = 1:length( FILESVOLTAGE )
    for B = 1:4096
        Ind = SpikesDynamic[ A ][ B ] .+ ( A - 1 )*binsize;
        AllInd[ B ] = vcat( AllInd[ B ], Ind );
    end
end

# for n = 1:4096
#     FILESPIKES = joinpath( PATHSPIKES, string( "ch", lpad( n, 4, "0" ), "_spikes.jld" ) )
#     spikes = zeros( nSpikesDynamic[ n ], SpikeSize );
#     save( FILESPIKES, "data", spikes )
# end

save(
    raw"C:\Users\Siby\Desktop\Datos\Hipocampo\Phase_10\Info\SpikesInfo.jld",
    "AllInd", AllInd,
    "nSpikesDynamic", nSpikesDynamic,
    "SpikesDynamic", SpikesDynamic )

MUAlF = Parameters[ "MUAlF" ];
MUAHF = Parameters[ "MUAHF" ];
MUAfac = Parameters[ "MUAfac" ];
A = 16#:length( FILESVOLTAGE )
FILEVOLTAGE = FILESVOLTAGE[ A ];
BIN = Float64.( LoadDict( FILEVOLTAGE ) );
BIN = round.( MatrixFilter( 
        Parameters, BIN, MUAlF, MUAHF, MUAfac, "BPF" ), digits = 3 );
Spikes = [ ];
Indexes = [ ];
for ch in 1:4096
    aux = SpikesDynamic[ A ][ ch ];
    push!( Indexes, aux );
    INI = aux .- ms2Frs( Parameters, Parameters[ "w_pre" ] );
    INI[ INI .<= 0 ] .= 1;
    ENDI = aux .+ ms2Frs( Parameters, Parameters[ "w_post" ] );
    ENDI[ ENDI .> size( BIN, 2 ) ] .= size( BIN, 2 );
    sp = [ ];
    for ie = 1:length( INI )
        push!( sp, BIN[ ch, INI[ ie ]:ENDI[ ie ] ] );
    end
    push!( Spikes, sp );
end
FILESPIKES = replace( FILEVOLTAGE, "Voltage" => "Spikes" );
save( FILESPIKES, "Spikes", Spikes, "Indexes", Indexes );

# # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# SpikesStatic = [ ];
# SpikesDynamic = [ ];

# for α = 1:length( FILESINDEXES );
#     aux = LoadDict( FILESINDEXES[ α ] );
#     MUASpikes = aux[ 2 ];
#     push!( SpikesStatic, MUASpikes[ 1 ] );
#     push!( SpikesDynamic, MUASpikes[ 2 ] );
# end
# # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# nSpikesDynamic = zeros( Int, 4096 );
# nSpikesStatic = zeros( Int, 4096 );
# for α in 1:length( FILESINDEXES );
#     nSpikesStatic = nSpikesStatic .+ length.( SpikesStatic[ α ] );
#     nSpikesDynamic = nSpikesDynamic .+ length.( SpikesDynamic[ α ] );
# end



# SpikeSize = ms2Frs( Parameters, w_pre ) + ms2Frs( Parameters, w_post ) + 1;

# channels = 1:size( BINMUA, 1 );
# # ValidChannels = findall( nSpikesStatic .!= 0 );
# # Spikes = Dict( );
# # for VCh in ValidChannels
# #     Spikes[ string( VCh ) ] = zeros( nSpikesStatic[ VCh ], SpikeSize );
# # end

# # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# for α = 1:length( FILESVOLTAGE )
#     FILEVOLTAGE = FILESVOLTAGE[ α ];
#     aux = LoadDict( FILEVOLTAGE );
#     # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
#     MUAlF = Parameters[ "MUAlF" ];
#     MUAHF = Parameters[ "MUAHF" ];
#     MUAfac = Parameters[ "MUAfac" ];
#     BINMUA = round.( MatrixFilter( 
#             Parameters, Float64.( aux ), MUAlF, MUAHF, MUAfac, "BPF" ), digits = 3 );
#     Spikes = [ ];
#     Indexes = [ ];
#     for ch in channels
#         aux = SpikesStatic[ α ][ ch ];
#         push!( Indexes, aux )
#         INI = aux .- ms2Frs( Parameters, w_pre );
#         INI[ INI .<= 0 ] .= 1;
#         ENDI = aux .+ ms2Frs( Parameters, w_post );
#         ENDI[ ENDI .> size( BINMUA, 2 ) ] .= size( BINMUA, 2 );
#         sp = [ ]
#         for ie = 1:length( INI )
#             push!( sp, BINMUA[ ch, INI[ ie ]:ENDI[ ie ] ] );
#         end
#         push!( Spikes, sp )
#     end
#     FILESPIKES = replace( FILEVOLTAGE, "Voltage" => "Spikes" );
#     save( FILESPIKES, "Spikes", Spikes, "Indexes", Indexes );
# end

# # FILESSPIKES = FindContent( "Spikes", FILEBRW );

# # AllSpikes = Array{ Any }( undef, 4096 ); fill!( AllSpikes, []);
# # AllIndexes = Array{ Any }( undef, 4096 ); fill!( AllIndexes, []);
# # @time for a = 1:length( FILESSPIKES )
# #     aux = load( FILESSPIKES[ a ] );
# #     Spikes = aux[ "Spikes" ];
# #     Indexes = aux[ "Indexes" ];
# #     for b = 1:4096
# #         AllSpikes[ b ] = vcat( AllSpikes[ b ], Spikes[ b ] );
# #         AllIndexes[ b ] = vcat( AllIndexes[ b ], Indexes[ b ] );
# #     end
# # end